# Assignment 2 starter code
This notebook contains code to run [coreferee](https://github.com/explosion/coreferee), a coreference system running under spaCy to extract coreference chains (or clusters) from text.
To run the notebook, you first have to intall coreferee. See instructions here: https://spacy.io/universe/project/coreferee, but mostly what you need to do is, from a command prompt:

    $ python -m pip install coreferee
$ python -m coreferee install en
    
You'll also need to download the spaCy transformer model for English, which needs the large model as well. 
It turns out, spacy has just released new versions and coreferee is not yet compatible with them, so you need to download specific versions of each model:

    $ python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.4.0/en_core_web_lg-3.4.0.tar.gz 
$ python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.4.0/en_core_web_trf-3.4.0.tar.gz

If you are running this on binder, then do these things in the next two cells instead

## Part 1: Run coreferee

### Binder only

In [ ]:
# run this cell only if you are on Binder

!python3 -m pip install coreferee
!python3 -m coreferee install en

In [ ]:
# run this cell only if you are on Binder

!python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.4.0/en_core_web_lg-3.4.0.tar.gz
!python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.4.0/en_core_web_trf-3.4.0.tar.gz

### End of Binder only

In [ ]:
# import what we need, load the transformer model, 
# add coreferee to the spacy nlp pipeline

import coreferee, spacy
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('coreferee')

In [ ]:
# this is just a test, so that you can see what the coreference chains look like
# you may get a CUDA warning here. As long as it's only a warning, things should run just fine

doc = nlp('Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much.')

In [ ]:
# now we print the coreference chains found

doc._.coref_chains.print()

A few things to note about the output:

* We have 4 coreference chains, relating to: *Peter, work, wife(+Peter), Spain*
* Coreferee is able to deal with cataphora, where the pronoun (*he*) appears before the referent (*Peter*)
* Coreferee can deal with groups: *\[he+wife\], they*
* The wife does not appear as an entity with a chain, because there is no referring expression to that entity. It only appears as part of *he and his wife*

In [ ]:
# once we have an index for a particular referring expression, 
# we can ask coreferee to resolve it. For instance, printing
# the following expression gives us the referent for 
# the referring expression 31 (they)

print(doc._.coref_chains.resolve(doc[31]))

### Run coreferee on local files

In [ ]:
# do coreference chains for 5 documents in the data/ directory
# below is a sample for the first text

with open ("data/5c1dbe1d1e67d78e2797d611.txt", "r", encoding='utf-8') as f:
    text1 = f.read()

In [ ]:
doc1 = nlp(text1)

In [ ]:
doc1._.coref_chains.print()

In [ ]:
# example: who does she(452) refer to?

print(doc1._.coref_chains.resolve(doc1[452]))

In [ ]:
# print referring expressions that are people
# we are interested in those because they are the sources of quotes
for ent in doc1.ents:
    if ent.label_ in ["PERSON"]:
        print(ent.text, ent.label_)

## Side note: visualizations
If you want to see this all in a much prettier format, you can use [displacy](https://spacy.io/usage/visualizers). 

In [ ]:
from spacy import displacy

In [ ]:
options = {"ents": ["PERSON"],
          "colors": {"PERSON": "lightsteelblue"}}

displacy.render(doc1, style="ent", options=options, jupyter=True)

## Part 2: Run the quote extraction from Assignment 1
I suggest using the Matcher quote extraction system from A1, but, if you implemented your own version, or improved on this one, feel free to use that instead.

In [ ]:
#import what we need for this
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

In [ ]:
# we don't need to load the text again; use text1 from above

matcher = Matcher(nlp.vocab)
pattern_q = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}]
matcher.add("QUOTES", [pattern_q], greedy='LONGEST')
doc = nlp(text1)
matches_q = matcher(doc)
matches_q.sort(key = lambda x: x[1])
print (len(matches_q))
for match in matches_q[:10]:
    print (match, doc[match[1]:match[2]])

## Your turn

Check instructions on Canvas for what to do and what to submit. 